In [6]:
from ase.build import bulk, make_supercell
from ase.io.vasp import write_vasp
from ase.io.lammpsdata import write_lammps_data


def hcp_gen():
    struc = bulk("Zr", "hcp", a=3.324, c=5.168, orthorhombic=False)
    struc_orth = bulk("Zr", "hcp", a=3.324, c=5.168, orthorhombic=True)
    print(struc)

    write_vasp("POSCAR", struc, direct=True, sort=True)
    write_vasp("POSCAR_orth", struc_orth, direct=True, sort=True)
    write_lammps_data("data.pos", struc, units="metal", atom_style="atomic")  # 没有写入质量mass信息


if __name__ == "__main__":
    hcp_gen()

Atoms(symbols='Zr2', pbc=True, cell=[[3.324, 0.0, 0.0], [-1.662, 2.8786684421794737, 0.0], [0.0, 0.0, 5.168]])


In [7]:
import numpy as np
from ase.build import bulk
from pymatgen.io.ase import AseAtomsAdaptor

struc = bulk("Zr", "hcp", a=3.234, c=5.168, orthorhombic=False)
struc.positions
type(struc.positions)
print(np.round(struc.positions, 7))

print(struc.get_positions())

struc_pmg = AseAtomsAdaptor.get_structure(struc)
struc_pmg

[[ 0.         0.         0.       ]
 [-0.         1.8671508  2.584    ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-8.81887156e-17  1.86715077e+00  2.58400000e+00]]


Structure Summary
Lattice
    abc : 3.234 3.2339999999999995 5.168
 angles : 90.0 90.0 120.00000000000001
 volume : 46.809410069095726
      A : 3.234 0.0 0.0
      B : -1.617 2.8007261558388743 0.0
      C : 0.0 0.0 5.168
    pbc : True True True
PeriodicSite: Zr (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Zr (0.0000, 1.8672, 2.5840) [0.3333, 0.6667, 0.5000]

ase atoms类与pymatgen structure类的互相转换

In [2]:
from ase.build import bulk
from pymatgen.io.ase import AseAtomsAdaptor


struc_ase = bulk('Fe', 'bcc', a=2.83, cubic=True)
struc_pmg = AseAtomsAdaptor.get_structure(struc_ase)

print(struc_ase)
print("\n")
print(struc_pmg)

Atoms(symbols='Fe2', pbc=True, cell=[2.83, 2.83, 2.83])


Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.830000   2.830000   2.830000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


ase中lammps文件格式: lammps-data（有read和write）和lammps-dump-text（只有read）、lammps-dump-binary lammps（只有read）

In [8]:
# showfile("data.pos")

In [13]:
from ase.io.lammpsdata import read_lammps_data, write_lammps_data
from ase.io.lammpsrun import read_lammps_dump_text, read_lammps_dump_binary


# ase io lammps related file
# symbol isn't right
struc = read_lammps_data('data.pos', style='atomic', units='metal')
# write_lammps_data('5.lammps-data', struc)

print(struc)
print(struc.get_atomic_numbers())
print(struc.get_chemical_symbols())

Atoms(symbols='H4', pbc=True, cell=[4.05, 4.05, 4.05], id=..., masses=..., type=...)
[1 1 1 1]
['H', 'H', 'H', 'H']
